<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>

# Backpropagation Practice

## *Data Science Unit 4 Sprint 2 Assignment 2*

Implement a 3 input, 4 node hidden-layer, 1 output node Multilayer Perceptron on the following dataset:

| x1 | x2 | x3 | y |
|----|----|----|---|
| 0  | 0  | 1  | 0 |
| 0  | 1  | 1  | 1 |
| 1  | 0  | 1  | 1 |
| 0  | 1  | 0  | 1 |
| 1  | 0  | 0  | 1 |
| 1  | 1  | 1  | 0 |
| 0  | 0  | 0  | 0 |

If you look at the data you'll notice that the first two columns behave like an XOR gate while the last column is mostly just noise. Remember that creating an XOR gate was what the perceptron was criticized for not being able to learn. 

In [64]:
import numpy as np

In [119]:
import pandas as pd
data = {
    'x1': [0,0,1,0,1,1,0],
    'x2': [0,1,0,1,0,1,0],
    'x3': [1,1,1,0,0,1,0],
    'y': [0,1,1,1,1,0,0]
}
df = pd.DataFrame.from_dict(data)
df

,x1,x2,x3,y
0,0,0,1,0
1,0,1,1,1
2,1,0,1,1
3,0,1,0,1
4,1,0,0,1
5,1,1,1,0
6,0,0,0,0


In [120]:
X = np.array(df[['x1','x2']].values)
y = np.array(df['y'].values)

In [121]:
class NeuralNetwork:
    def __init__(self):
        self.inputs = 2 # Set up Architecture of Neural Network
        self.hiddenNodes = 3
        self.outputNodes = 1 #regression problem
        
        self.weights1 = np.random.randn(self.inputs, self.hiddenNodes)#initalize weights, 2x3  matrix array for first layer
        #nodes from input to hidden layers
        
        #3x1 Matrix Array for Hidden to Output 
        self.weights2 = np.random.randn(self.hiddenNodes, self.outputNodes)
        #nodes from hidden nodes to input
nn = NeuralNetwork()
print("Layer 1 weights: \n", nn.weights1)
print("Layer 2 weights: \n", nn.weights2)

Layer 1 weights: 
 [[ 0.05578761 -0.20502679 -0.55785292]
 [-0.574949   -1.48010042  0.0244496 ]]
Layer 2 weights: 
 [[-0.81524754]
 [ 0.22031476]
 [-1.06912962]]


In [122]:
class NeuralNetwork:
    def __init__(self):
        self.inputs = 2 # Set up Architecture of Neural Network
        self.hiddenNodes = 3
        self.outputNodes = 1
        
        self.weights1 = np.random.randn(self.inputs, self.hiddenNodes)#initalize weights, 2x3  matrix array for first layer
        
        #3x1 Matrix Array for Hidden to Output 
        self.weights2 = np.random.randn(self.hiddenNodes, self.outputNodes)
        
        #need sigmoid for activation functions
        #need bias terms and fixed function error metric
    def sigmoid(self, s):
        return 1 / (1+np.exp(-s))
    
    def feed_forward(self, X):
        """
        Calculate the NN inference using feed forward pass.
        """
#input to hidden, hidden to output        
        #Weighted sum of inputs and hidden layers, dot product of X and self weights1
        self.hidden_sum = np.dot(X, self.weights1)#with hidden sum, we calculate and return for activation function 
        
        #Activation of the weighted sum
        self.activated_hidden = self.sigmoid(self.hidden_sum)#activated hidden to the output layer
        
        #Weighted sum of the hidden layer to output layer
        self.output_sum = np.dot(self.activated_hidden, self.weights2)
        
        #Final Activation of Output
        self.activated_output = self.sigmoid(self.output_sum)
        
        return self.activated_output
    
nn = NeuralNetwork()

print(X[0])
output = nn.feed_forward(X[0])#make a feedforward pass
print("output", output)

[0 0]
output [0.26685878]


In [123]:
error = y[0] - output #actual activation - ouput
error

array([-0.26685878])

In [124]:
output_all = nn.feed_forward(X)
error_all = y - output_all
print(output_all)
print(error_all)

[[0.26685878]
 [0.35827283]
 [0.18890694]
 [0.35827283]
 [0.18890694]
 [0.26977302]
 [0.26685878]]
[[-0.26685878  0.73314122  0.73314122  0.73314122  0.73314122 -0.26685878
  -0.26685878]
 [-0.35827283  0.64172717  0.64172717  0.64172717  0.64172717 -0.35827283
  -0.35827283]
 [-0.18890694  0.81109306  0.81109306  0.81109306  0.81109306 -0.18890694
  -0.18890694]
 [-0.35827283  0.64172717  0.64172717  0.64172717  0.64172717 -0.35827283
  -0.35827283]
 [-0.18890694  0.81109306  0.81109306  0.81109306  0.81109306 -0.18890694
  -0.18890694]
 [-0.26977302  0.73022698  0.73022698  0.73022698  0.73022698 -0.26977302
  -0.26977302]
 [-0.26685878  0.73314122  0.73314122  0.73314122  0.73314122 -0.26685878
  -0.26685878]]


In [125]:
attributes = ['weights1', 'hidden_sum', 'activated_hidden', 'weights2', 'output+']
[print(i+'\n', getattr(nn,i), '\n'+'---'*3) for i in dir(nn) if i[:2]!= '__']

activated_hidden
 [[0.5        0.5        0.5       ]
 [0.32384997 0.7026395  0.2121049 ]
 [0.06766202 0.55036754 0.74647645]
 [0.32384997 0.7026395  0.2121049 ]
 [0.06766202 0.55036754 0.74647645]
 [0.03359178 0.74308298 0.44216585]
 [0.5        0.5        0.5       ]] 
---------
activated_output
 [[0.26685878]
 [0.35827283]
 [0.18890694]
 [0.35827283]
 [0.18890694]
 [0.26977302]
 [0.26685878]] 
---------
feed_forward
 <bound method NeuralNetwork.feed_forward of <__main__.NeuralNetwork object at 0x00000249524967F0>> 
---------
hiddenNodes
 3 
---------
hidden_sum
 [[ 0.          0.          0.        ]
 [-0.73613464  0.85989874 -1.31228399]
 [-2.62317038  0.20215583  1.07990734]
 [-0.73613464  0.85989874 -1.31228399]
 [-2.62317038  0.20215583  1.07990734]
 [-3.35930501  1.06205458 -0.23237664]
 [ 0.          0.          0.        ]] 
---------
inputs
 2 
---------
outputNodes
 1 
---------
output_sum
 [[-1.01061875]
 [-0.58286845]
 [-1.45712826]
 [-0.58286845]
 [-1.45712826]
 [-0.9957

[None, None, None, None, None, None, None, None, None, None, None]

In [126]:
class NeuralNetwork:
    
    def __init__(self):
        # Set up Architecture of Neural Network
        self.inputs = 2
        self.hiddenNodes =  3
        self.outputNodes = 1

        # Initial Weights
        # 2x3 Matrix Array for the First Layer
        self.weights1 = np.random.randn(self.inputs, self.hiddenNodes)
       
        # 3x1 Matrix Array for Hidden to Output
        self.weights2 = np.random.randn(self.hiddenNodes, self.outputNodes)

    def sigmoid(self, s):
        return 1 / (1+np.exp(-s))

    def sigmoidPrime(self, s):
        return s * (1-s)

    def feed_forward(self, X):
        """
        Calculate the NN inference using feed forward pass.
        """
        #Weighted sum of inputs and hidden layer
        self.hidden_sum = np.dot(X, self.weights1)

        #Activation of the weighted sum
        self.activated_hidden = self.sigmoid(self.hidden_sum)

        #Weighted sum of hidden layer to output layer
        self.output_sum = np.dot(self.activated_hidden, self.weights2)

        #Final Activation of Output
        self.activated_output = self.sigmoid(self.output_sum)

        return self.activated_output

    #sigmoid needed at this step, will take input matrix, outputs and some other term for error
    def backward(self, X, y, o):
        #this is our backward pass
        #updating weights by fixed amount which we calculated using gradient derivative of our error
        self.o_error = y - o#error in output

        #delta gives us the adjustment from the output to the hidden
        #derivative of error
        #Size of Adjustment from hidden =>output
        self.o_delta = self.o_error * self.sigmoidPrime(o) #apply derivative of sigmoid to error#basically the gradient
        #the z2 error is how much we need to change the weights by
        #delta is constant for the backward propagation pass, then gets smaller for every other pass hopefully

        self.z2_error = self.o_delta.dot(self.weights2.T)#How much the hidden layer interprets as the error
        self.z2_delta = self.z2_error * self.sigmoidPrime(self.activated_hidden)#this is where you might scale down the learning rate
        #we are updating the weights a fixed amount by travelling up the gradient
        
        #going from right to left
        #Adjustments from the hidden to the ouput weights
        self.weights2 += self.activated_hidden.T.dot(self.o_delta)

        #Adjustment from input => hidden weights
        #taking how much we get wrong from the z2 error and z2 delta
        #weights1 is the first hidden error, we are working backwards
        self.weights1 += X.T.dot(self.z2_delta)#update the weights of the first layer by taking the dot products 
        #with self.z2_delta which is representative of a variable value of how much we need to update the weights.

        #we are not using gradient descent, we are updating the weights a fixed amount that we calculated using just the 
        #gradient derivative of our error
    def train(self, X, y):
        o = self.feed_forward(X)#calculate a forward path
        self.backward(X, y, o )#make a backwards pass
        
#Since you're never passing back a function, it only matters if you can find the derivative of the activation function
#we only care about the derivative of the activation function, the chain rule for the backprop inputs will always be a constant.
nn.weights2

array([[ 0.01569167],
       [-0.31783189],
       [-1.71909729]])

In [127]:
#we are transposing weights so that this and our error is going to be flat
nn.weights2.T

array([[ 0.01569167, -0.31783189, -1.71909729]])

In [128]:
error#flat

array([-0.26685878])

In [129]:
# We want activated that correspond to negative weights to be lower
# And we want more higher activation for positivie weights

nn = NeuralNetwork()
o_delta = error * nn.sigmoidPrime(output)

z2_error = o_delta.dot(nn.weights2.T)
z2_error#set this error aside, calculate this error for the next layer, take the error all at once, then update the weights
#based on the size of the updates
#we want the array this outputs to be the same shape as our weights

array([ 0.00404219, -0.0061897 , -0.02716636])

In [130]:
nn = NeuralNetwork()
for i in range(1000):#1000 iterations on our model
    if (i+1 in [1,2,3,4,5]) or ((i+1) % 200 ==0):
        print('+' + '---' * 3 + f'EPOCH {i+1}' + '---'*3 + '+')
        print('Input: \n', X)
        print('Actual Output: \n', y)
        print('Predicted Output: \n', str(nn.feed_forward(X)))
        print("Loss: \n", str(np.mean(np.square(y - nn.feed_forward(X)))))
    nn.train(X,y)
    
#some neurons in the hidden layer are more highlighted than others because with backpropagation

+---------EPOCH 1---------+
Input: 
 [[0 0]
 [0 1]
 [1 0]
 [0 1]
 [1 0]
 [1 1]
 [0 0]]
Actual Output: 
 [0 1 1 1 1 0 0]
Predicted Output: 
 [[0.60113633]
 [0.61832852]
 [0.60805514]
 [0.61832852]
 [0.60805514]
 [0.63050647]
 [0.60113633]]
Loss: 
 0.24666046221404625


ValueError: shapes (7,7) and (1,3) not aligned: 7 (dim 1) != 1 (dim 0)

## Try building/training a more complex MLP on a bigger dataset.

Use the [MNIST dataset](http://yann.lecun.com/exdb/mnist/) to build the cannonical handwriting digit recognizer and see what kind of accuracy you can achieve. 

If you need inspiration, the internet is chalk-full of tutorials, but I want you to see how far you can get on your own first. I've linked to the original MNIST dataset above but it will probably be easier to download data through a neural network library. If you reference outside resources make sure you understand every line of code that you're using from other sources, and share with your fellow students helpful resources that you find.


## Stretch Goals: 

- Implement Cross Validation model evaluation on your MNIST implementation 
- Research different [Gradient Descent Based Optimizers](https://keras.io/optimizers/)
 - [Siraj Raval the evolution of gradient descent](https://www.youtube.com/watch?v=nhqo0u1a6fw)
- Build a housing price estimation model using a neural network. How does its accuracy compare with the regression models that we fit earlier on in class?